In [ ]:
import os
import random
import time
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.legacy import data, datasets

import random
import time
import spacy
import numpy as np
from torch import Tensor

In [ ]:
TEXT = data.Field(include_lengths=True)

LABEL = data.LabelField(dtype = torch.float) 

In [ ]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print('Number of training examples: {}'.format(len(train_data)))
print('Number of validation examples: {}'.format(len(valid_data)))
print('Number of testing examples: {}'.format(len(test_data)))

In [ ]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:0" if USE_CUDA else "cpu")

In [ ]:
!nvidia-smi

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_                 
                 )
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

In [ ]:
BATCH_SIZE = 32   
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_sizes = (BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
    sort_within_batch = True,
    device = device)

In [ ]:
class PositionalEnc(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class TransformerNet(nn.Module):    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_heads, n_layers, dropout, pad_idx):
        super().__init__()

        self.hidden_him = hidden_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)

        self.pe = PositionalEnc(embedding_dim) 

        enc_layer = nn.TransformerEncoderLayer(embedding_dim, n_heads, hidden_dim, dropout=dropout)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers)
      
        self.fc = nn.Linear(embedding_dim, output_dim)  

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        embedded = self.embedding(text)

        pos_encoded = self.pe(embedded)
        trans_out = self.encoder(pos_encoded)

        
        pooled = trans_out.mean(0)
        final = self.fc(self.dropout(pooled))

        return final



In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = 1
N_HEADS = 2 
N_LAYERS = 1
DROPOUT = 0.5

PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = TransformerNet(INPUT_DIM,      
                       EMBEDDING_DIM, 
                       HIDDEN_DIM, 
                       OUTPUT_DIM, 
                       N_HEADS,
                       N_LAYERS, 
                       DROPOUT,
                       PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('The model has {:,} trainable parameters'.format(count_parameters(model)))

pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings);

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model= model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text[0]).squeeze(1)  
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            predictions = model(batch.text[0]).squeeze(1) 
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
N_EPOCHS = 7

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'trans-model.pt')
    
    print('Epoch: {:02}'.format(epoch+1))
    print('\tTrain Loss: {:.3f} | Train Acc: {:.2f}%'.format(train_loss, train_acc*100))
    print('\t Val. Loss: {:.3f} |  Val. Acc: {:.2f}%'.format(valid_loss, valid_acc*100))

Epoch: 01
	Train Loss: 0.533 | Train Acc: 71.47%
	 Val. Loss: 0.431 |  Val. Acc: 82.02%
Epoch: 02
	Train Loss: 0.351 | Train Acc: 84.90%
	 Val. Loss: 0.438 |  Val. Acc: 82.77%
Epoch: 03
	Train Loss: 0.261 | Train Acc: 89.53%
	 Val. Loss: 0.376 |  Val. Acc: 86.57%
Epoch: 04
	Train Loss: 0.190 | Train Acc: 92.76%
	 Val. Loss: 0.483 |  Val. Acc: 86.23%
Epoch: 05
	Train Loss: 0.139 | Train Acc: 94.83%
	 Val. Loss: 0.510 |  Val. Acc: 87.01%
Epoch: 06
	Train Loss: 0.101 | Train Acc: 96.29%
	 Val. Loss: 0.653 |  Val. Acc: 86.21%
Epoch: 07
	Train Loss: 0.078 | Train Acc: 97.23%
	 Val. Loss: 0.742 |  Val. Acc: 86.05%


In [ ]:
model.load_state_dict(torch.load('trans-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print('Test Loss: {:.3f} | Test Acc: {:.2f}%'.format(test_loss, test_acc*100))

Test Loss: 0.386 | Test Acc: 86.50%


In [ ]:
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    print(tokenized)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    print(indexed)
    tensor = torch.LongTensor(indexed).to(device)
    print(tensor.shape)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is terrible") 

['This', 'film', 'is', 'terrible']
[49, 24, 7, 535]
torch.Size([4])


0.0006468613864853978

In [ ]:
predict_sentiment(model, "This film is terrible but great")

['This', 'film', 'is', 'terrible', 'but', 'great']
[49, 24, 7, 535, 18, 95]
torch.Size([6])


0.421206533908844